## DoEgen Synrift Example

<img src=".\images\doegen_exp_design.png" alt="drawing" width="50%"/>

***set up DoEgen Excel configuration file prior to this***

In [ ]:
#install the badlands_doe_toolset
!pip install -i https://test.pypi.org/simple/ badlands-doe-toolset


In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # deprecation warnings from DoEgen are supressed
import os
import contextlib

import doegen.doegen as doegen
import doegen.doeval as doeval

import badlands_doe_toolset.csv_to_badlands as genxml #DoEgen badlands csv to xml generation 
import badlands_doe_toolset.postproc_utils as ppu
import badlands_doe_toolset.well_doe_similarity as wellsim
import badlands_doe_toolset.badlands_multiproc_run as mpr

## Run DoEgen main process ##

In [ ]:
# Run the main DoEgen process to build the experiment design tables and evaluate inputs
# redirect output to log file
# calls DoEgen and loads the excel file with configuration parameters
with open('doegen_log.txt', 'w') as f:                  
    with contextlib.redirect_stdout(f):
        doegen.main('Experiment_setup_badlands.xlsx',\
            path='.', \
            outpath='design_results/', \
            nrun_max=300, \
            maxtime_per_run=120, \
            delta_nrun=4)                               

# at the end of the process, print the design result summary
with open('./design_results/Experiment_Design_selection_summary.txt') as f:
    b = f.read()
    print (b)
    


## Generate the Badlands experiment configuration files.

In [ ]:
# This creates the models xml files using the selected DoEgen Design Table (csv file)
# select the optimal or best design table from the DoEgen outputs in design_results.
# all of the experiment / model configuration files are written to the "badlands_xml" directory
genxml.main('design_results/Designtable_optimal_Nrun120.csv')

## Run the Badlands experiments in parallel


In [ ]:
# specify the location of the suite of experiment configuratio files and the optimal number of experiments to run in parallel.
experiment_run=mpr.badlands_multi_experiment('badlands_xml', proc=4)

## Post Processing
***Interpolate to final stratal file***


In [ ]:
# Select the attributes present in the tin.time.XX.hdf5 files that you want to interpolate onto the final stratal grid.
attributes=['lake','cumdiff']
ppu.strat_tin_write_doegen('badlands_xml',attributes)

## A similar function is available for calculated properties 
# attributes = ['fetch']
# ppu.strat_flow_write_doegen('badlands_xml',attributes)

## Extract data from psuedo well locations in all experiments


In [2]:
# note well location file format is required, see example.
Experiment=ppu.extract_WellLayer_all('badlands_xml', 'data/well_location_model.csv',outfile='experiment_well_extracts.hdf5')


Checking XML files are valid. If there is an error after this then there is something wrong with your configuration xml.
All good, simple xml check passed, number of experiment configuration files loaded: 90
load data from: Experiments/experiment_1/h5/sed.time300.hdf5


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'Experiments/experiment_1/h5/sed.time300.hdf5', errno = 2, error message = 'No such file or directory', flags = 1, o_flags = 2)

## Compute observed to experimental similarity comparison value and generate DoEval input files


In [ ]:
# This requires a few input parameters so I've split out the parameters here to make it more obvious.

model_extract='results_final/experiment_well_extracts.hdf5' # the model extracts hdf5 file.
wells_file = 'data/well_location_model.csv' # the location of the file with information about all wells.

obslog='GR' # the name of the log/observation column in the csv file.
expattrib='Lake' # the name of the attibute in the hdf5 extracted values
minthkRatio=0.8 # experimental thickness / measured thickness cutoff ratio for evaluating the obslog/expattrib similarity.

analysis_results=wellsim.exp_attr_sim_DoEgen(model_extract,obslog,expattrib,wells_file,minthkRatio,exp_srm=5,log_smth=10,outdir='results_final')


## DoEval on results

In [ ]:
# the design table used to generate the experiment configurations.
designtable='design_results/Designtable_best_Nrun120.csv'

# doeval.main(initial_directory,DoEval file with attribute to be evaluated, the designtable, the directory to output the results to)
# This produces DoEvaloutputs based on the ratio of experimental/measured well thicknesses.
doeval.main('.','./results_final/doeval_well_thickRatio.csv',designtable,'results_final/well_thickness/')

# This evaluates the well similarities.
doeval.main('.','./results_final/doeval_well_similarity.csv',designtable,'results_final/well_similarity/')


## Review results


In [ ]:
# Lets have a look at some of the results.
# The results for the Joll01 attribute/log similarity can be viewed
Joll01_sim=pd.read_csv('results_final/well_similarity/Experiment_Joll01_RMSE_Top10_sorted.csv')
Joll01_sim

## Plot measured vs. experimental attributes

The plots from this are saved to the **results_final/plots** folder.

In [ ]:
#The well_attr_plot function plots can be produced at any time 
wellfile = 'data/well_location_model.csv'
#model_extract=h5py.File(psuedowells,'r')
model_extract='results_final/experiment_well_extracts.hdf5'
well_log_file='data/Jolly1_GR.csv' # this is the location of the file with the measured / log property
obslog='GR' #this is name of the column in the measured log or well property to be compared to the experiments

prefix="experiment_"
explog='Lake' # the name of the attribute in the extracted experiment file
Well='Joll01' # the well name string from the well location file.

for i in Joll01_sim['Nexp']:
    Exp=prefix+str(i)
    #print(wellsim.well_attr_sim(model_extract,Exp,Well,obslog,well_log_file,explog,exp_srm=5,log_smth=10))
    wellsim.well_attr_plot(model_extract,Exp,Well,obslog,well_log_file,explog,exp_srm=5,log_smth=10)
    

In [ ]:
#Depending on the number of experiments run it may be useful to plot all of the results.
model_extract='results_final/experiment_well_extracts.hdf5'
wellfile = 'data/well_location_model.csv'
wellsim.well_attr_plot_DoEgen(model_extract,'GR',wellfile,'Lake')

In [ ]:
# Or only the results for one experiment
model_extract='results_final/experiment_well_extracts.hdf5'
wellfile = 'data/well_location_model.csv'

wellsim.exp_attr_plot(model_extract,'experiment_111','GR',wellfile,'Lake')

